# Import Library

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

In [7]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

# Crawling data melalui PTA UTM

In [9]:
# Daftar prodi Fakultas Teknik
PRODI_FATEK = {
    10: "Teknik Informatika",
    11: "Teknik Industri",
    12: "Teknik Mesin",
    13: "Teknik Elektro",
    14: "Teknik Mekatronika",
    15: "Sistem Informasi",
    16: "Teknik Multimedia dan Jaringan",
    17: "Manajemen Informatika",
    18: "Mekatronika"
}

# Batas maksimal halaman per prodi (biar tidak lama)
MAX_PAGE_LIMIT = 15   # <-- ganti jadi 10 kalau mau lebih banyak

def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        max_page = int(href.split("/")[-1])
        return max_page
    return 1

def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

def pta_fak_teknik():
    start_time = time.time()

    data = {
        "id": [], "penulis": [], "judul": [], "abstrak_id": [],
        "abstrak_en": [], "pembimbing_pertama": [], "pembimbing_kedua": [],
        "prodi": []
    }

    for prodi_id, prodi_name in PRODI_FATEK.items():
        max_page = min(get_max_page(prodi_id), MAX_PAGE_LIMIT)  # batasi halaman
        for j in range(1, max_page + 1):
            url = f"{BASE_URL}/{prodi_id}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            for jurnal in jurnals:
                link_keluar = jurnal.select_one('a.gray.button')['href']
                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip() if isi.select_one('a.title') else "N/A"
                penulis = isi.find(string=re.compile("Penulis")) if isi else "N/A"
                pembimbing_pertama = isi.find(string=re.compile("Dosen Pembimbing I")) if isi else "N/A"
                pembimbing_kedua = isi.find(string=re.compile("Dosen Pembimbing II")) if isi else "N/A"

                paragraf = isi.select('p[align="justify"]') if isi else []
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi_name)

            print_progress(prodi_id, prodi_name, j, max_page)

        sys.stdout.write("\n")

    # simpan hasil
    df = pd.DataFrame(data)
    df.to_csv("pta_fak_teknik.csv", index=False, encoding="utf-8-sig")

    elapsed = int(time.time() - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Data Fakultas Teknik berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

# Hasil Crawling data

In [10]:
pta_fak_teknik()

[10] Teknik Informatika - Page 15/15 [████████████████████] 100.00%

[11] Teknik Industri - Page 15/15 [████████████████████] 100.00%

[12] Teknik Mesin - Page 15/15 [████████████████████] 100.00%

[13] Teknik Elektro - Page 15/15 [████████████████████] 100.00%

[14] Teknik Mekatronika - Page 15/15 [████████████████████] 100.00%

[15] Sistem Informasi - Page 15/15 [████████████████████] 100.00%

[16] Teknik Multimedia dan Jaringan - Page 15/15 [████████████████████] 100.00%

[17] Manajemen Informatika - Page 15/15 [████████████████████] 100.00%

[18] Mekatronika - Page 15/15 [████████████████████] 100.00%


✅ Data Fakultas Teknik berhasil dikumpulkan!
📊 Total entri: 675
⏱️ Waktu eksekusi: 1 jam 21 menit 20 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,040411100468,Penulis : A.Ubaidillah S.Kom,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,Academic information systems (SIAKAD) is an in...,Dosen Pembimbing I : Budi Setyono M.T,Dosen Pembimbing II :Hermawan S.T,Teknik Informatika
1,040411100476,"Penulis : M. Basith Ardianto,",APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,-,"Dosen Pembimbing I : Drs. Budi Soesilo, MT","Dosen Pembimbing II :Koko Joni, ST",Teknik Informatika
2,040411100480,"Penulis : Akhmad Suyandi, S.Kom",RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,Web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...,"Dosen Pembimbing I : Drs. Budi Soesilo, M.T","Dosen Pembimbing II :Hermawan, ST, MT",Teknik Informatika
3,070411100070,Penulis : Heri Supriyanto,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Scheduling courses in universities is a ...,"Dosen Pembimbing I : Mulaab, S.Si., M.Kom","Dosen Pembimbing II :Firli Irhamni, ST., M.Kom",Teknik Informatika
4,080411100115,Penulis : Septian Rahman Hakim,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Haryanto, S.T., M.T.",Teknik Informatika
...,...,...,...,...,...,...,...,...
670,110611100137,Penulis : Enik Ningsih,Perbandingan Hasil Belajar Siswa Menggunakan M...,Penelitian ini bertujuan untuk mengetahui apak...,The purpose of this study to determine whether...,"Dosen Pembimbing I : Mohammad Edy Nurtamam, S....",Dosen Pembimbing II :Ariesta Kartika Sari S.Si...,Mekatronika
671,110611100111,Penulis : Titis Kurnia Eka Fajariesta,PENGARUH TEMAN SEBAYA TERHADAP KEMAMPUAN KOGNI...,Latar belakang penelitian ini adalah Setiap ke...,Background of this study was that every learni...,"Dosen Pembimbing I : H. Priyono Tri Febrianto,...","Dosen Pembimbing II :Muhammad Busyro Karim, S....",Mekatronika
672,110611100112,Penulis : MARIA ULFA,PENGARUH METODE MIND MAPPING TERHADAP HASIL BE...,Permasalahan yang terjadi saat ini adalah hasi...,The problem happen in this moment is a still l...,"Dosen Pembimbing I : Eva Ari Wahyuni, S.Pd., M.Si","Dosen Pembimbing II :Muhammad Busyro Karim, S....",Mekatronika
673,110611100083,Penulis : Rista Apriliya Devi,PENGEMBANGAN PERANGKAT PEMBELAJARAN TEMATIK BE...,Perangkat pembelajaran merupakan serangkaian b...,"Learning tools is a series of materials, equip...","Dosen Pembimbing I : Hani’ah, S.Pd., M.Pd.","Dosen Pembimbing II :Mujtahidin, S.Pd., M.Pd.",Mekatronika


# Crawling Link keluaran

In [11]:
# Progress bar
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

# Crawling link page + link keluar
def pta_links():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "no": [],
        "page": [],
        "link_keluar": []
    }

    no = 1  # nomor urut

    for i in range(1, 42):  # looping semua prodi (id 1–41)
        total_pages = 3  # bisa diubah sesuai kebutuhan (misal 10 halaman / semua halaman)
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop halaman per prodi
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                data["no"].append(no)
                data["page"].append(url)          # link halaman
                data["link_keluar"].append(link)  # link detail TA
                no += 1

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta_links.csv", index=False)

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [12]:
pta_links()

[1] Ilmu Hukum - Page 3/3 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 3/3 [████████████████████] 100.00%

[3] Agribisnis - Page 3/3 [████████████████████] 100.00%

[4] Agroteknologi - Page 3/3 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 3/3 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 3/3 [████████████████████] 100.00%

[7] Manajemen - Page 3/3 [████████████████████] 100.00%

[8] Akuntansi - Page 3/3 [████████████████████] 100.00%

[9] Teknik Industri - Page 3/3 [████████████████████] 100.00%

[10] Teknik Informatika - Page 3/3 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 3/3 [████████████████████] 100.00%

[12] Sosiologi - Page 3/3 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 3/3 [████████████████████] 100.00%

[14] Psikologi - Page 3/3 [████████████████████] 100.00%

[15] Sastra Inggris - Page 3/3 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 3/3 [████████████████████] 100.00%

,no,page,link_keluar
0,1,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
1,2,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
2,3,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
3,4,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/090...
4,5,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
...,...,...,...
476,477,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
477,478,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
478,479,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...
479,480,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...


# Crawling Berita Melalui Website detik.com

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [14]:

# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"


In [15]:
#Fungsi untuk membersihkan konten artikel
def clean_content(content_element):
    if not content_element:
        return "Content Not Found"
    return " ".join(content_element.stripped_strings)

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # berhenti jika artikel sudah cukup

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = clean_content(content_element)

        judul.append(title)
        isi.append(content)
        kategori_list.append(kategori)

        # tampilkan 10 judul pertama saja sebagai indikator
        if len(judul) <= 100:
            print(title)

# URL kategori Detik
base_urls = [
    "https://news.detik.com/indeks",
    "https://sport.detik.com/indeks",
]

categories = [
    "Politik",
    "Olahraga",
]

# Inisialisasi list untuk menyimpan data
judul = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 100

# Looping setiap kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}?page={page}"   # ✅ perbaikan format URL
        get_data(url, category, min_articles_per_category)
        time.sleep(2)
        page += 1

# Buat dataframe
df = pd.DataFrame({"judul": judul, "isi": isi, "kategori": kategori_list})

# Simpan ke CSV
df.to_csv("CrawlBerita_Politik-Olahraga.csv", index=False, encoding="utf-8-sig")

print("✅ Crawling selesai, data disimpan ke CrawlBerita_Politik-Olahraga.csv")


KPK Spill Pucuk Pimpinan Kemenag Terima Dana Korupsi Kuota Haji
Istana Apresiasi Transformasi Nusakambangan: Kolaborasi untuk Visi-Misi Presiden
Kapolda Riau Puji Tim Raga Polres Meranti: Garda Terdepan Penjaga Keamanan
Ini Alasan KPK Periksa Ustaz Khalid Basalamah di Kasus Korupsi Kuota Haji
Menag Sebut Bakal Ada Lembaga Tampung Dana Umat buat Atasi Kemiskinan
Menag Doakan Kementerian Haji Raih Survei Kepuasan Jemaah di Atas 90%
Video: Pihak Keluarga dr Aulia Nilai Tuntutan Para Terdakwa Terlalu Ringan
Israel Serang Markas Komando di Ibu Kota Yaman
Video: Polri Bantu Evakuasi Warga Terdampak Banjir dan Longsor di Bali
Marcella Santoso Ngaku Diancam Terdakwa Suap Migor: Ada Kata Pasang Leher
Peringati HUT ke-80, TNI AL Gelar Bakti Sosial di Ponpes Bogor
Polri Salurkan 1.500 Paket Bansos ke Warga Kukar Kaltim
Tanggul Beton di Utara Jakarta Tuai Tanda Tanya
Survei BPS: Tingkat Kepuasan Jemaah Haji RI 2025 88,46%, Naik dari Tahun Lalu
Legislator Harap Tata Kelola Haji Lebih Terintegrasi, 

In [16]:
df=pd.read_csv("CrawlBerita_Politik-Olahraga.csv")
df.head(200)

,judul,isi,kategori
0,KPK Spill Pucuk Pimpinan Kemenag Terima Dana K...,Jakarta - KPK mengungkap adanya dugaan aliran ...,Politik
1,Istana Apresiasi Transformasi Nusakambangan: K...,Cilacap - Kantor Komunikasi Kepresidenan atau ...,Politik
2,Kapolda Riau Puji Tim Raga Polres Meranti: Gar...,"Jakarta - Kapolda Riau, Irjen Pol Herry Heryaw...",Politik
3,Ini Alasan KPK Periksa Ustaz Khalid Basalamah ...,Jakarta - KPK telah memeriksa Ustaz Khalid Zee...,Politik
4,Menag Sebut Bakal Ada Lembaga Tampung Dana Uma...,Jakarta - Menteri Agama (Menag) Nasaruddin Uma...,Politik
...,...,...,...
195,Semarak Bendera Palestina dalam Balap Sepeda d...,Bilbao - Penonton mengibarkan bendera Palestin...,Olahraga
196,"Tatap MotoGP Catalunya, Francesco Bagnaia Suda...","Jakarta - Rider Ducati , Francesco Bagnaia , o...",Olahraga
197,Komang Ayu Cahya Dewi Mundur dari Pelatnas PBSI,Jakarta - Komang Ayu Cahya Dewi mundur dari pe...,Olahraga
198,Penghormatan untuk Kobe Bryant pada Mamba Day ...,Jakarta - Project 95 (P95) Basketball Academy ...,Olahraga
